# DSCI 100 Project Final Report (Group 40) 
> Github: [dsci-100-project](https://github.com/wizexplorer/dsci-100-project)

## Predicting Newsletter Subscription from Player Behaviour

In [ ]:
# Load required packages
library(tidyverse)
library(repr)
library(tidymodels)

# set seed for reproducability
set.seed(0)

options(repr.plot.width = 12, repr.plot.height = 8)

### Introduction

#### Background:
Game companies often use newsletters to keep players engaged with
new content, events, and promotions. However, sending messages
to players who are unlikely to subscribe or interact can waste
marketing effort. If we can predict which players are more likely
to subscribe, the company can target those users more efficiently
and potentially improve engagement and revenue.

#### Research questions:
##### Broad question:
What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter?

##### Specific question:
Can we accurately predict whether a player subscribes to the game-related newsletter based on their age, gender, experience level, total hours played, and in-game behavior (average session duration, number of sessions)?   
  
Accuracy metric will not be very useful here because the response class is not balanced.. "TRUE" observations outnumber the "FALSE" once by 3 times (shown later). Hence, we will focus on improving the recall of the model as we believe missing out on potential subscribers (recall) is more costly than targeting uninterested players (precision).   

#### Data description:
We use two datasets `players`  and  `sessions`:


__Players dataset__   
The players dataset contains 196 observations of players with 7 attributes:  
  - `experience` (character): Self-reported experience level of the player
  - `subscribe`(logical): Whether player subscribed to the newsletter
  - `hashedEmail` (character): Hashed unique player identifier
  - `played_hours` (double): Total number of hours played by the player
  - `name` (character): Player's chosen in-game name
  - `gender` (character): Player's gender
  - `Age` (double): Player's age in years

At first we see that the data is tidy, but to make it usable for our purposes (classification), we will need to convert the character types to categorical variables (factor):

__Sessions dataset__   
The sessions dataset contains 1535 observations of players' sessions with 5 attributes:  
  - `hashedEmail` (character): Hashed unique player identifier
  - `start_time` (character): Human-readable session start time (format: DD/MM/YYYY HH:MM)
  - `end_time` (character): Human-readable session end time (format: DD/MM/YYYY HH:MM)
  - `original_start_time` (double): Session start time as Unix timestamp (milliseconds)
  - `original_end_time` (double): Session end time as Unix timestamp (milliseconds)

This data is almost tidy! Although, `start_time` and `end_time` could be separated into two variables each, we choose not to do so in this case, because we already have a combined (absolute) version for the times which we can use instead of these variables. Instead, we convert them into datetime objects to compute statistics on the dataset. Also, these variables may be helpful to convert the ineligible Unix time into legible human understandable time later.

From the sessions data, we will derive per-player engagement metrics:
  - `mean_session_duration` (double): average session length in minutes
  - `total_session_duration` (double): total minutes played across sessions
  - `num_sessions` (integer): number of recorded sessions

#### Aim:
Our aim is to build a k-nearest neighbours (k-NN) classifier that predicts subscribe using:   
  Age, experience, gender, played_hours,   
  mean_session_duration, total_session_duration, num_sessions,   
and missingness indicators for the session variables. We will use
5-fold cross-validation to tune k and focus on recall.


### Methododology & Results

In this section we will:
  1) Load and clean the data.
  2) Create derived session features and join to players.
  3) Explore the data with summaries and visualizations.
  4) Build and tune a k-NN model with tidymodels.
  5) Evaluate the final model on a test set.

__1.1 Load the data__

In [ ]:
players <- read_csv("https://raw.githubusercontent.com/wizexplorer/dsci-100-project/main/data/players.csv")
sessions <- read_csv("https://raw.githubusercontent.com/wizexplorer/dsci-100-project/main/data/sessions.csv")

In [ ]:
# Quick dimension checks
dim(players)
dim(sessions)

__1.2 Clean the data__

In [ ]:
players <- players |>
  mutate(
    subscribe = as.factor(subscribe),
    experience = as.factor(experience),
    gender = as.factor(gender)
  )

sessions <- sessions |>
  mutate(
    start_time = dmy_hm(start_time),
    end_time = dmy_hm(end_time),
    session_duration_mins = as.numeric(difftime(end_time, start_time, units = "mins"))
  )

__2.1 Aggregate sessions to player level__   
For each player (hashedEmail) we compute:
  - mean_session_duration
  - total_session_duration
  - num_sessions

In [ ]:
session_by_player <- sessions |>
  group_by(hashedEmail) |>
  summarise(
    mean_session_duration = mean(session_duration_mins, na.rm = TRUE),
    total_session_duration = sum(session_duration_mins, na.rm = TRUE),
    num_sessions = n()
  )

__2.2 Join the aggregated sessions data with players dataset__

In [ ]:
data <- left_join(x=players, y=session_by_player, by = "hashedEmail")
glimpse(data)

In [ ]:
missing_data <- data |>
  summarise(
    n_missing_age = sum(is.na(Age)),
    n_missing_played_hours = sum(is.na(played_hours)),
    n_missing_mean_session = sum(is.na(mean_session_duration)),
    n_missing_total_session = sum(is.na(total_session_duration)),
    n_missing_num_session = sum(is.na(num_sessions))
  )
missing_data

Notice, we have missing data for 71 players from the sessions dataset. Now, 71 observations are a considerable portion of our dataset so we cannot afford the k-nn model removing these rows during training. Consequently, we must somehow make the model include these rows.

We can do this in two ways:
- Impute missing values with either a `0` or `mean` of variable.
- Impute missing values with `0` and create a missing indicator variable to let the model learn if the absence of data carries a signal.

Since we already decided to choose predictive variables using cross validation, we will go with option 2. If the missing data doesn't correspond to a signal, it will automatically have a low accuracy and not be choosen as a pridictive variable.   

We will also impute the 2 missing `Age` observations with its `mean` since this likely does not have a missing information and even if it does, 2 observations might not justify creating an additional column and likely won't affect our accuracy too much.   

Finally, we reorder the factor levels of our response variable `subscribe`.

In [ ]:
data <- data |>
  mutate(
    # Create missing indicators (as factors for modeling)
    missing_mean_duration = as.factor(is.na(mean_session_duration)),
    missing_total_duration = as.factor(is.na(total_session_duration)),
    missing_num_sessions   = as.factor(is.na(num_sessions)),

    # Impute missing values with 0 (assumes no session activity)
    mean_session_duration  = replace_na(mean_session_duration, 0),
    total_session_duration = replace_na(total_session_duration, 0),
    num_sessions           = replace_na(num_sessions, 0),

    # Impute with mean
    Age = if_else(
      is.na(Age),
      mean(Age, na.rm = TRUE),
      Age
    ),
    # Finally, reorder factor levels for later
    subscribe = fct_relevel(subscribe, "TRUE")
  )
glimpse(data)

Check if we have solved the missing variable's issue:

In [ ]:
missing_data <- data |>
  summarise(
    n_missing_age = sum(is.na(Age)),
    n_missing_played_hours = sum(is.na(played_hours)),
    n_missing_mean_session = sum(is.na(mean_session_duration)),
    n_missing_total_session = sum(is.na(total_session_duration)),
    n_missing_num_session = sum(is.na(num_sessions))
  )
missing_data

__3 Exploratory Data Analysis__

We summarize statistics on the numeric data

In [ ]:
summary <- data |>
  group_by(subscribe) |>
  summarise(
    mean_played_hours        = mean(played_hours, na.rm = TRUE),
    mean_age                 = mean(Age, na.rm = TRUE),
    mean_mean_session        = mean(mean_session_duration, na.rm = TRUE),
    mean_total_session       = mean(total_session_duration, na.rm = TRUE),
    mean_num_sessions        = mean(num_sessions, na.rm = TRUE),
    n                        = n(),
    .groups = "drop"
  )

summary

We also note class balance of the response variable:

In [ ]:
data |>
  count(subscribe) |>
  mutate(prop = n / sum(n))

Although the response classes are somewhat imbalanced (26.5% non-subscribers vs 73.5% subscribers), we will not perform re-sampling. 
This is because our primary goal is to correctly identify subscribers (the majority class), so we actually want the model to be generous in predicting TRUE, not to give FALSE extra weight. Upsampling the FALSE or downsampling the TRUE here would mostly protect the non-subscriber class and could reduce recall for TRUE, which is opposite of our goal.

In [ ]:
data |>
  ggplot(aes(x = Age, fill = subscribe)) +
  geom_histogram(position = "dodge", binwidth = 2, alpha = 0.7) +
  labs(
    title = "Figure 1. Age Distribution by Subscription",
    x = "Age", y = "Count",
    fill = "Subscribed"
  ) +
  theme(text = element_text(size=18))

From the summaries and Figure 1, we observe that:
- Ages cluster around early 20s, and players closer to 20
  appear somewhat more likely to subscribe.

In [ ]:
data |>
  group_by(subscribe) |>
  summarise(avg_sessions = mean(num_sessions)) |>
  ggplot(aes(x = subscribe, y = avg_sessions, fill = subscribe)) +
  geom_col() +
  labs(
    title = "Figure 2. Average Number of Sessions by Subscription Status",
    x = "Subscription status", y = "Average Number of Sessions",
    fill = "Subscribed"
  ) +
  theme(text = element_text(size=18))

From the summaries and Figure 2, we observe that:
- Higher number of sessions and longer session durations tend to be
  associated with higher subscription rates.

In [ ]:
data |>
  group_by(subscribe) |>
  summarise(avg_played_hours = mean(played_hours, na.rm = TRUE)) |>
  ggplot(aes(x = subscribe, y = avg_played_hours, fill = subscribe)) +
  geom_col() +
  labs(
    title = "Figure 3. Average Played Hours by Subscription Status",
    x = "Subscription", y = "Average Played Hours",
    fill = "Subscribed"
  ) +
  theme(text = element_text(size=18))

From the summaries and Figure 3, we observe that:
- Higher played_hours and longer session durations tend to be
  associated with higher subscription rates.

In [ ]:
players |>
  ggplot(aes(x = experience, fill = subscribe)) +
  geom_bar(position = "fill") +
  labs(title = "Figure 4. Experience Level to Subscription Status", x = "Experience Level", y = "Proportion Of Players (%)", fill = "Subscribed") +
  theme(text = element_text(size=18))

From the information presented in Figure 4, we observe that:

- Across all experience levels, the percentage of subscribed stays within the range of approximately 65-80%.
- Overall, the differences in experience level has a minor impact on whether the player subscribes to the in-game newsletter or not.

In [ ]:
summary |>
    ggplot(aes(x = subscribe, y = mean_mean_session, fill = subscribe)) +
    geom_col(alpha = 0.7) +
    labs(title = "Figure 5. Average Session Duration by Subscription Status", x = "Subscribed Count", y = "Average Session Duration", fill = "Subscribed") +
    theme(text = element_text(size=18))

From the information presented in Figure 5, we observe that:

- The higher the average session duration, the more likely that the player will subscribe to the in-game newsletter.

These patterns motivate using both demographic and behavioural
predictors in our classification model.

__4 Modelling the dataset using k-NN classification__

To build a k-NN classifier using tidymodels. We:
  - Split the data into training and test sets (80/20 stratified).
  - Define a recipe to normalise numeric predictors and create
    dummy variables for categorical predictors.
  - Tune k using 5-fold cross-validation, focusing on recall.
  - Select the best k, fit the final model, and evaluate it on
    the test set.

__4.1 Split the data__

In [ ]:
data_split <- initial_split(data, prop = 0.8, strata = subscribe)
train_data <- training(data_split)
test_data  <- testing(data_split)

__4.2 Cross-validation folds on the training set__

In [ ]:
folds <- vfold_cv(train_data, v = 5, strata = subscribe)

__4.3 Define the recipe for preprocessing__

In [ ]:
knn_recipe <- recipe(
  subscribe ~ Age + experience + gender + played_hours +
    mean_session_duration + total_session_duration + num_sessions +
    missing_mean_duration + missing_total_duration + missing_num_sessions,
  data = train_data
) |>
  step_center(all_numeric_predictors()) |>
  step_scale(all_numeric_predictors()) |>
  step_dummy(all_nominal_predictors())

__4.4 Define the k-NN model specification__

In [ ]:
knn_spec <- nearest_neighbor(
  mode      = "classification",
  neighbors = tune()
) |>
  set_engine("kknn")

__4.5 Combine recipe and model into a workflow__

In [ ]:
knn_workflow <- workflow() |>
  add_model(knn_spec) |>
  add_recipe(knn_recipe)

__4.6 Define k range and tune via 5-fold cross-validation__

In [ ]:
knn_grid <- tibble(
  neighbors = seq(1, 40, by = 2)
)

knn_tune <- tune_grid(
  knn_workflow,
  resamples = folds,
  grid      = knn_grid,
  metrics   = metric_set(recall, accuracy, precision)
    )
knn_results <- knn_tune |> collect_metrics()

knn_results

Plot the results of cross-validation

In [ ]:
# recall_vs_k_lots <- ggplot(knn_results, aes(x = neighbors, y = mean)) +
#   geom_point() +
#   geom_line() +
#   labs(x = "Neighbors", y = "Recall Estimate") +
#   theme(text = element_text(size = 18))

# recall_vs_k_lots

autoplot(knn_tune) + 
    theme(text = element_text(size=18)) +
    ggtitle("Figure 4. Neighbours vs Performance Estimates")


We can see from the plot that around k = 33, recall reaches 100% but this is due to the fact that the majority of our data consists of `TRUE` predictions. At this k, the `TRUE` observations overpower the `FALSE` observations at every point hence the model always chooses `TRUE` and gets a 100% recall. So, model isn't really learning the pattern. Hence, we use the other accuracy estimates to choose the best k as well.   

Note that the best results occur between the range k = 15 and k = 30, so we examine and choose a k in this range with the highest total accuracy across all metrics. To do this, we first wrangle the results tibble and add all the metrics (in proportion to their contribution we need) in a `total_metrics` column and choose the k with the best `total_metrics`. After some tries, we find that the combination of 10% each for accuracy and recall and 80% for precision gives us the best k. (This was done through trial and error and hence not shown).   

We needed to reduce recall and accuracy's contribution due to the class imbalance in the data we discussed earlier. Both accuracy and recall would benifit from just predicting `TRUE` most of the time since the dataset is biased towards this response. Precission however would not benifit as much as it would lose from predicting a false positive (`FALSE` predicted as `TRUE`)  which is why we need it to have a greater influence on the `total_metrics`.

In [ ]:
metrics <- knn_results |>
            select(neighbors, .metric, mean) |>
            pivot_wider(
                names_from = .metric,
                values_from = mean
            ) |>
            filter(neighbors >= 15 & neighbors <= 30) |>
            mutate(
                total_metrics = accuracy*0.10 + precision*0.8 + recall*0.10
            ) |>
            arrange(desc(total_metrics))
print(metrics, n = 10)

We store the best k in `best_k`

In [ ]:
best_k <- metrics |>
        head(1) |>
        pull(neighbors)
best_k

__4.7 Train the final model with best k__

In [ ]:
best_spec <- nearest_neighbor(
  mode      = "classification",
  neighbors = best_k
) |>
  set_engine("kknn")

best_fit <- workflow() |>
                    add_model(best_spec) |>
                    add_recipe(knn_recipe) |>
                    fit(data = train_data)

best_fit

__5 Evaluate the model on test set__

In [ ]:
test_predictions <- predict(best_fit, test_data) |>
                    bind_cols(test_data)
test_predictions

We now calculate the model's performance:

In [ ]:
test_accuracy <- test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

test_precision <- test_predictions |>
    precision(truth = subscribe, estimate = .pred_class, event_level="first")

test_recall <- test_predictions |>
    recall(truth = subscribe, estimate = .pred_class, event_level="first")

confusion <- test_predictions |>
             conf_mat(truth = subscribe, estimate = .pred_class)

confusion
test_accuracy
test_precision
test_recall

The output shows that the estimated accuracy, precision and recall of the classifier on the test data was 75%, 74% and 100%, respectively.   

The confusion matrix shows 29 observations were correctly predicted as TRUE, and 1 were correctly predicted as FALSE. It also shows that the classifier made some mistakes; in particular, it classified 10 observations as TRUE when they were actually FALSE. One good thing is that correctly identified all TRUE values, i.e. no observation was predicted as FALSE it was actually TRUE. This matches with our high recall strategy.

### Discussion

#### Summary
We built a k-NN classifier to predict newsletter subscription from a combination of demographic (Age, gender, experience) and behavioural (played_hours and session-based engagement) features.   

After aggregating session data by player, handling missing session values with both imputation and missingness indicators, and standardising numeric predictors, we tuned k using 5-fold cross-validation and selected the value that maximised recall without completely sacrificing accuracy and precisioin. This was done so that the model retains perfect recall for subscribers while making at least some distinction between subscribers and non-subscribers.   

On the test set, the final model achieves a moderate overall performance (with 75% accuracy and 74% precision) and, crucially, a perfect (100%) recall for the response variable i.e. `subscriber` class. Our model successfully recovers a meaningful fraction of actual subscribers while accepting some false positives.

#### Relation to Expectations

Our final k-NN model with the chosen k achieved recall = 1 for the subscriber (TRUE) class, accuracy of 0.75, and precision of about 0.74 on the test set.
This means the model successfully identified all actual subscribers in the test data, which aligns with our original expectation that behavioural and
demographic features (such as experience level, played hours, and session duration) carry useful signal for predicting subscription. We expected (from  Figure 1, 2 and 3) that more engaged and more experienced players would be easier to classify as subscribers, and the perfect recall for TRUE is consistent with this idea.

At the same time, the magnitude of the improvement over the trivial baseline (predicting every player as a subscriber) was modest: the baseline accuracy is roughly 0.73 (since 73% of the dataset represents the TRUE response), and our model improves this only slightly to 0.75 while still misclassifying most non-subscribers (correctly identifying only 1 out of 11).  

This partially contradicts any expectation that we would achieve a model that clearly separates subscribers from non-subscribers; instead, our results show that, given a small and imbalanced dataset, k-NN struggles to learn a strong pattern for the minority non-subscriber class.   

Overall, the model behaves in line with our priority of maximising recall for subscribers, but it confirms that distinguishing non-subscribers is considerably more difficult than we initially hoped.

#### Potential Impact

In practice, this model could help a game studio target players who are more likely to subscribe to newsletters or other communications. Because we prioritise recall, the model is designed to capture most potential subscribers, even if some non-subscribers are mistakenly contacted. For a marketing team, this may be acceptable if the cost of extra emails is low relative to the value of a new engaged subscriber (which we're assuming is).

#### Future Directions
Our model achieves perfect recall for subscribers but only slightly improves on the trivial “everyone subscribes” baseline and still rarely identifies non-subscribers correctly. Future work could:
  - Try alternative models (e.g. clustering) that may better separate non-subscribers while keeping high recall.
  - Explicitly handle class imbalance (e.g., re-sampling or class weights) to give the minority non-subscriber class more influence in training.

Overall, our analysis supports the idea that both demographic characteristics (age, gender, total played hours, experience) and in-game behaviour contain useful signal for predicting newsletter subscription, although the prediction problem remains challenging.

*__TODO: add better conclusion__*

In [ ]:
Conclusion